<a href="https://colab.research.google.com/github/LawrenceLLY/GNN_Pun_Detection/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.0 MB/s eta 0:00:00


In [ ]:
!wget https://alt.qcri.org/semeval2017/task7/data/uploads/semeval2017_task7.tar.xz
!tar -xf semeval2017_task7.tar.xz
#!tar -xvf semeval2017_task7.tar.xz
#%cd semeval2017_task7/
#%cd ..
%ls

--2023-05-02 20:31:11--  https://alt.qcri.org/semeval2017/task7/data/uploads/semeval2017_task7.tar.xz
Resolving alt.qcri.org (alt.qcri.org)... 80.76.166.231
Connecting to alt.qcri.org (alt.qcri.org)|80.76.166.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 748424 (731K) [application/x-xz]
Saving to: ‘semeval2017_task7.tar.xz’

semeval2017_task7.t 100%[===================>] 730.88K   819KB/s    in 0.9s    

2023-05-02 20:31:13 (819 KB/s) - ‘semeval2017_task7.tar.xz’ saved [748424/748424]

sample_data/  semeval2017_task7/  semeval2017_task7.tar.xz


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import numpy as np
import copy
import xml.etree.ElementTree as ET

In [ ]:
f = 'semeval2017_task7/data/test/subtask2-heterographic-test.xml'

mytree = ET.parse(f)
myroot = mytree.getroot()

puns = []
for item in myroot.findall('./text'):
  dict1 = {}
  dict1[item.attrib['id']] = {}
  for child in item:
    idd = child.attrib['id']
    dict1[item.attrib['id']][idd] = child.text
  puns.append(dict1)

print(puns[0])

{'het_1': {'het_1_1': "'", 'het_1_2': "'", 'het_1_3': 'I', 'het_1_4': "'", 'het_1_5': 'm', 'het_1_6': 'halfway', 'het_1_7': 'up', 'het_1_8': 'a', 'het_1_9': 'mountain', 'het_1_10': ',', 'het_1_11': "'", 'het_1_12': "'", 'het_1_13': 'Tom', 'het_1_14': 'alleged', 'het_1_15': '.'}}


In [ ]:
gold = []
with open('semeval2017_task7/data/test/subtask2-heterographic-test.gold', 'r') as fin:
  for row in fin:
    gold.append(row.strip().split('\t')[1])
print(gold[:5])

['het_1_14', 'het_2_13', 'het_4_11', 'het_5_5', 'het_7_6']


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased", output_hidden_states=True).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def bert(puns, ratio=0):
    res = []
    for i in tqdm(puns):
        for pun in i.values():
            poss = [x for x in pun]
            token_num = []
            start = False
            for x in pun:
                subwords = tokenizer.tokenize((' ' if start else '') + pun[x])
                token_num.append(len(subwords))
                start = True
            sentence = ' '.join([pun[x] for x in pun])
            tokenized = tokenizer(sentence, return_tensors="pt").to(device)
            # tokenizer will automatic add the start token (ID: 101) and end token (ID: 102) of the sentence
            # The token ID of "[MASK]" is 103
            # #token >= #word
            word_probabilities_list = []
            for mask_i in range(1, len(tokenized['input_ids'][0]) - 1):
                tokenized_input = (copy.deepcopy(tokenized)).to(device)
                tokenized_input['input_ids'][0][mask_i] = torch.tensor(103) # The ID of "[MASK]" is 103
                outputs = model(**tokenized_input)
                logits = outputs.logits.to(device)
                old_word_ID = tokenized['input_ids'][0][mask_i]
                prob = torch.nn.functional.softmax(logits[0][mask_i], dim=-1)[old_word_ID].item()
                word_probabilities_list.append(prob)
        final_word_probabilities_list = np.ones((len(poss),), dtype=np.float64)
        start_tok = 0
        for word_index, num_token in enumerate(token_num):
            curr = 1.0
            for each_tok_i in range(num_token):
                curr *= word_probabilities_list[start_tok]
                start_tok += 1
            final_word_probabilities_list[word_index] = curr
        start_index = min(len(poss) - 1, int(ratio * len(poss)))
        min_index = start_index + np.argmin(final_word_probabilities_list[start_index:], axis=0)
        res.append(poss[min_index])
    return res

In [ ]:
bert(puns[30:33], 0.6)

100%|██████████| 3/3 [00:08<00:00,  2.74s/it]


['het_42_11', 'het_43_11', 'het_44_14']

In [ ]:
print("accuracy_score:", accuracy_score(gold, bert(puns, ratio=0.6)))

  0%|          | 1/1271 [00:07<2:34:16,  7.29s/it]


KeyboardInterrupt: ignored

In [ ]:
tokenizer.tokenize("pretrain")

['pre', '##train']

In [ ]:
tokenizer.tokenize("ChatGPT")

['chat', '##gp', '##t']